## APC--Area per capita

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import math
import os

print(sys.executable)
print(os.environ.get('CONDA_DEFAULT_ENV', '未检测到Conda环境'))
!conda env list  # 列出所有环境，当前环境前会标有 *
!conda info      # 查看详细信息，包括当前环境
!jupyter kernelspec list  # 列出所有内核

/opt/miniconda3/envs/mixb/bin/python
mixb

# conda environments:
#
base                   /opt/miniconda3
flwr                   /opt/miniconda3/envs/flwr
geospatial             /opt/miniconda3/envs/geospatial
mixb                 * /opt/miniconda3/envs/mixb


     active environment : mixb
    active env location : /opt/miniconda3/envs/mixb
            shell level : 2
       user config file : /Users/apple/.condarc
 populated config files : /opt/miniconda3/.condarc
                          /Users/apple/.condarc
          conda version : 25.3.1
    conda-build version : not installed
         python version : 3.13.2.final.0
                 solver : libmamba (default)
       virtual packages : __archspec=1=skylake
                          __conda=25.3.1=0
                          __osx=13.4=0
                          __unix=0=0
       base environment : /opt/miniconda3  (writable)
      conda av data dir : /opt/miniconda3/etc/conda
  conda av metadata url : None
           channel 

In [2]:
!jupyter kernelspec list          # 查看所有 kernel
# !jupyter kernelspec remove myenv

Available kernels:
  python3    /opt/miniconda3/envs/mixb/share/jupyter/kernels/python3
  mixb       /Users/apple/Library/Jupyter/kernels/mixb


## 1. 数据预处理

In [3]:
def load_and_melt(file_path, value_name):
    """
    加载CSV文件并检查年份列中的无效值
    
    参数:
        file_path (str): CSV文件路径
        value_name (str): 要创建的指标名称
    
    返回:
        DataFrame: 处理后的数据框
    """
    # 读取CSV文件
    df = pd.read_csv(file_path)
    
    # 清理未命名列
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    # 转换为长格式
    melted = df.melt(
        id_vars='Countries',
        var_name='Year',
        value_name=value_name
    )
    
    # 尝试将年份转换为数值，无效值设为NaN
    melted['Year_numeric'] = pd.to_numeric(melted['Year'], errors='coerce')
    
    # 查找无效年份值
    invalid_mask = melted['Year_numeric'].isna()
    invalid_rows = melted[invalid_mask]
    
    # 如果有无效值，打印详细信息
    if not invalid_rows.empty:
        print(f"\n⚠️ 在文件 {file_path} 中发现无效年份值:")
        for idx, row in invalid_rows.iterrows():
            print(f"  行索引: {idx}, 国家: '{row['Countries']}', 无效年份值: '{row['Year']}'")
        print(f"共发现 {len(invalid_rows)} 个无效年份值\n")
    
    # 删除无效行并转换年份为整数
    melted = melted.dropna(subset=['Year_numeric'])
    melted['Year'] = melted['Year_numeric'].astype(int)
    melted = melted.drop(columns=['Year_numeric'])
    
    return melted

'''
def load_and_melt(file_path, value_name):
    """
    读取文件并转换为长格式（Country, Year, Value）
    """
    # 读取文件，第一列为国家名称
    df = pd.read_csv(file_path)   
    # 转换为长格式
    melted = df.melt(
        id_vars='Countries',
        var_name='Year',
        value_name=value_name
    )
    melted['Year'] = melted['Year'].astype(int)  # 确保年份为整数
    return melted
'''

'\ndef load_and_melt(file_path, value_name):\n    """\n    读取文件并转换为长格式（Country, Year, Value）\n    """\n    # 读取文件，第一列为国家名称\n    df = pd.read_csv(file_path)   \n    # 转换为长格式\n    melted = df.melt(\n        id_vars=\'Countries\',\n        var_name=\'Year\',\n        value_name=value_name\n    )\n    melted[\'Year\'] = melted[\'Year\'].astype(int)  # 确保年份为整数\n    return melted\n'

In [4]:
# 定义文件路径及其对应的指标名
file_config = {
    'Data1/processed/CO2EmissionsFromEnergy.csv': 'total_CE',
    'Data1/processed/CarbonSequestration.csv': 'cement_CS',
    'Data1/processed/PrimaryEnergyConsumption.csv': 'energy_consumption',
    'Data1/processed/Population.csv': 'Population',
    'Data1/processed/GDP.csv': 'GDP',
    'Data1/processed/CementProduction.csv': 'cement_production',
    'Data1/processed/SurfaceArea.csv': 'built_surface'
}

In [5]:
# 示例：加载数据
for file, col_name in file_config.items():
    f_df = load_and_melt(file, col_name)
    print(f_df[f_df["Countries"]=="China"].head(4))

    Countries  Year total_CE
76      China  1965    488.5
168     China  1966    530.3
260     China  1967    475.9
352     China  1968    476.7
    Countries  Year  cement_CS
36      China  1928   0.034032
163     China  1928   0.034032
208     China  1929   0.054537
335     China  1929   0.054537
    Countries  Year energy_consumption
76      China  1965               5.53
168     China  1966               6.01
260     China  1967                5.4
352     China  1968               5.46
    Countries  Year   Population
39      China  1960  667070000.0
254     China  1961  660330000.0
469     China  1962  665770000.0
684     China  1963  682335000.0
    Countries  Year           GDP
39      China  1960  5.971625e+10
254     China  1961  5.005669e+10
469     China  1962  4.720919e+10
684     China  1963  5.070661e+10
    Countries  Year  cement_production
36      China  1928         252.000000
199     China  1929         369.000000
362     China  1930         451.613505
525     China 

In [9]:
# 初始化合并后的 DataFrame
merged_df = None
previous_countries = set()  # 用于跟踪国家列表变化

# 逐个加载并合并
for file, col_name in file_config.items():
    print(f"处理文件: {file}")
    temp_df = load_and_melt(file, col_name)  # 使用修改后的函数
    
    if merged_df is None:
        merged_df = temp_df
        previous_countries = set(merged_df['Countries'].unique())
        print(f"  初始数据框大小: {merged_df.shape}")
        print(f"  包含国家数量: {len(previous_countries)}")
    else:
        before_merge_countries = set(merged_df['Countries'].unique())
        before_merge_rows = merged_df.shape[0]
        
        merged_df = pd.merge(
            merged_df,
            temp_df, 
            on=['Countries', 'Year'],  # 注意列名是'Countries'而不是'Country'
            how='inner'
        )
        
        after_merge_countries = set(merged_df['Countries'].unique())
        after_merge_rows = merged_df.shape[0]
        
        # 计算减少的国家数量
        lost_countries = before_merge_countries - after_merge_countries
        num_lost = len(lost_countries)
        
        print(f"  合并后数据框大小: {merged_df.shape}, 减少行数: {before_merge_rows - after_merge_rows}")
        print(f"  减少国家数量: {num_lost}")
        if num_lost > 0:
            # print(f"  丢失的国家示例: {list(lost_countries)[:5]}...")  # 只显示前5个示例
            print(f"  丢失的国家列表: {list(lost_countries)[:]}")
            
        previous_countries = after_merge_countries

# 按国家和年份排序
merged_df = merged_df.sort_values(['Countries', 'Year']).reset_index(drop=True)
print("\n✅ 所有文件处理完成!")
print(f"最终数据框大小: {merged_df.shape}")
print(f"最终包含国家数量: {len(merged_df['Countries'].unique())}")

# --- 新增: 添加 region 信息 ---
# 从 GDP 或 Population 文件中提取 region 信息（假设这些文件有 region 列）
# 这里以 GDP 文件为例

# 首先加载 GDP 文件并提取国家和 region 的映射
gdp_df = pd.read_csv('Data1/processed/GDP.csv')

# 假设 GDP 文件中有 'Country' 和 'region' 列（根据实际情况调整列名）
# 我们需要确保每个国家只有一个 region，所以取第一个非空值
iso = pd.read_csv('Data1/processed/iso.csv',index_col=0).reset_index()
region_map = iso.dropna(subset=['region']).drop_duplicates('name')[['name', 'region']]

# 将 region 信息合并到最终数据框
# 注意：确保 'Countries' 列名与 merged_df 匹配
merged_df = pd.merge(
    merged_df,
    region_map,
    left_on='Countries',
    right_on='name',
    how='left'
).drop(columns='name')  # 删除多余的列

# 检查哪些国家没有 region 信息
countries_without_region = merged_df[merged_df['region'].isna()]['Countries'].unique()
if len(countries_without_region) > 0:
    print(f"\n⚠️ 以下国家缺少 region 信息: {list(countries_without_region)}")
    # 可以选择用默认值填充
    # merged_df['region'] = merged_df['region'].fillna('Unknown')

print("\n✅ region 信息添加完成!")
print(merged_df.head())

处理文件: Data1/processed/CO2EmissionsFromEnergy.csv
  初始数据框大小: (5336, 3)
  包含国家数量: 92
处理文件: Data1/processed/CarbonSequestration.csv
  合并后数据框大小: (4814, 4), 减少行数: 522
  减少国家数量: 14
  丢失的国家列表: ['Other Caribbean', 'Other Europe', 'Other Northern Africa', 'Trinidad & Tobago', 'Other Southern Africa', 'Other South America', 'Central America', 'Other Middle East', 'Other CIS', 'Western Africa', 'Middle Africa', 'Eastern Africa', 'Other Asia Pacific', 'USSR']
处理文件: Data1/processed/PrimaryEnergyConsumption.csv
  合并后数据框大小: (4814, 5), 减少行数: 0
  减少国家数量: 0
处理文件: Data1/processed/Population.csv
  合并后数据框大小: (4756, 6), 减少行数: 58
  减少国家数量: 1
  丢失的国家列表: ['Taiwan']
处理文件: Data1/processed/GDP.csv
  合并后数据框大小: (4698, 7), 减少行数: 58
  减少国家数量: 1
  丢失的国家列表: ['Turkey']
处理文件: Data1/processed/CementProduction.csv
  合并后数据框大小: (4640, 8), 减少行数: 58
  减少国家数量: 1
  丢失的国家列表: ['South Korea']
处理文件: Data1/processed/SurfaceArea.csv
  合并后数据框大小: (800, 9), 减少行数: 3840
  减少国家数量: 0

✅ 所有文件处理完成!
最终数据框大小: (800, 9)
最终包含国家数量: 75

⚠️ 以下国家缺少 reg

In [10]:
# --- 新增: 筛选1975年之后的数据 ---
before_filter = merged_df.shape[0]
before_filter_countries = set(merged_df['Countries'].unique())
merged_df = merged_df[merged_df['Year'] >= 1975]
after_filter_countries = set(merged_df['Countries'].unique())

lost_countries_filter = before_filter_countries - after_filter_countries
num_lost_filter = len(lost_countries_filter)

print(f"\n筛选1975年之后的数据: {merged_df.shape}")
print(f"因年份筛选减少的国家数量: {num_lost_filter}")
if num_lost_filter > 0:
    # print(f"因年份筛选丢失的国家示例: {list(lost_countries_filter)[:5]}...")
    print(f"因年份筛选丢失的国家列表: {list(lost_countries_filter)[:]}")

# --- 新增: 去除国家名和年份都重复的数据 ---
before_dedup = merged_df.shape[0]
merged_df = merged_df.drop_duplicates(subset=['Countries', 'Year'], keep='first')
after_dedup = merged_df.shape[0]
dup_count = before_dedup - after_dedup

if dup_count > 0:
    print(f"\n⚠️ 发现并移除了 {dup_count} 行国家名和年份重复的数据")
else:
    print("\n✅ 没有发现国家名和年份重复的数据")
print(f"去重后数据框大小: {merged_df.shape}")

# 将所有数值列转换为浮点数（为后续除法计算）
numeric_cols = [
    'total_CE', 'cement_CS', 'energy_consumption', 'Population', 
    'GDP', 'cement_production', 'built_surface'
]

# 定义极小值常量
SMALL_VALUE = 1e-10

for col in numeric_cols:
    # 转换并替换无效值为极小值
    initial_valid = merged_df[col].notna().sum()
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
    
    # 统计无效值数量
    invalid_count = merged_df[col].isna().sum()
    
    if invalid_count > 0:
        # 将NaN替换为极小值
        merged_df[col] = merged_df[col].fillna(SMALL_VALUE)
        print(f"⚠️ 列 '{col}' 中发现 {invalid_count} 个无效值，已替换为极小值 ({SMALL_VALUE})")
    else:
        print(f"✅ 列 '{col}' 无无效值")

# 显示中国数据
print("\n中国数据示例:")
print(merged_df[merged_df["Countries"]=="China"].head())


筛选1975年之后的数据: (800, 10)
因年份筛选减少的国家数量: 0

⚠️ 发现并移除了 50 行国家名和年份重复的数据
去重后数据框大小: (750, 10)
⚠️ 列 'total_CE' 中发现 29 个无效值，已替换为极小值 (1e-10)
✅ 列 'cement_CS' 无无效值
⚠️ 列 'energy_consumption' 中发现 29 个无效值，已替换为极小值 (1e-10)
✅ 列 'Population' 无无效值
⚠️ 列 'GDP' 中发现 67 个无效值，已替换为极小值 (1e-10)
✅ 列 'cement_production' 无无效值
✅ 列 'built_surface' 无无效值

中国数据示例:
    Countries  Year  total_CE  cement_CS  energy_consumption    Population  \
130     China  1975    1131.3   5.143255               13.25  9.163950e+08   
132     China  1980    1460.9  14.516760               17.38  9.812350e+08   
134     China  1985    1821.7  20.384433               22.14  1.051040e+09   
136     China  1990    2308.8  28.138505               28.58  1.135185e+09   
138     China  1995    3009.2  56.739027               37.27  1.204855e+09   

              GDP  cement_production  built_surface region  
130  1.634300e+11          30021.700   2.000345e+10   Asia  
132  1.911490e+11          95442.703   2.423702e+10   Asia  
134  3.094880e+11

## 2. 因素的构建与操作

### 2.1 计算因素

In [11]:
# 数据整理
# 定义分解所需的因子列名
# APC更新后的因子列表
factors = [
    'total_CE/energy_consumption',
    'energy_consumption/GDP',
    'GDP/Population',
    'built_surface/Population',  # 修改为建筑面积/人口
    'cement_CS/built_surface' # 单位建筑面积的吸收量
]
# 在计算分解因子前添加单位转换
print("执行单位转换...")
# 质量单位转换 (水泥产量: kt → Mt) 百万吨
merged_df['cement_production'] = merged_df['cement_production'] / 1000
# 经济单位转换 (GDP: USD → trillion USD) 万亿美元
merged_df['GDP'] = merged_df['GDP'] / 1e12
# 人口单位转换 (persons → billion persons) 人口(亿人)
merged_df['Population'] = merged_df['Population'] / 1e8
# 现在计算分解因子（使用统一单位）
print("计算分解因子...")

# 确保所有关键列均为正数（避免除以0/对非正值取对数）
merged_df = merged_df[
    (merged_df['total_CE'] > 0) &
    (merged_df['cement_CS'] > 0) &
    (merged_df['built_surface'] > 0) &
    (merged_df['cement_production'] > 0) &
    (merged_df['GDP'] > 0) &
    (merged_df['Population'] > 0)&
    (merged_df['energy_consumption'] > 0) 
]

# 计算各分解因子
merged_df['total_CE/energy_consumption'] = merged_df['total_CE'] / merged_df['energy_consumption']
merged_df['energy_consumption/GDP'] = merged_df['energy_consumption'] / merged_df['GDP']
merged_df['GDP/Population'] = merged_df['GDP'] / merged_df['Population']
# APC修改后的因子计算
merged_df['built_surface/Population'] = merged_df['built_surface'] / merged_df['Population']
# CS per Area修改后因子计算
merged_df['cement_CS/built_surface'] = merged_df['cement_CS'] / merged_df['built_surface']
# 加入cfp
merged_df['CFP'] = merged_df['total_CE'] / merged_df['cement_CS']

# 再次检查无穷大和缺失值
# print("清理无穷值和缺失值...")
# merged_df = merged_df.replace([np.inf, -np.inf], np.nan)
# merged_df = merged_df.dropna(subset=factors + ['total_CE', 'cement_CS'])
print(f"最终有效数据量: {len(merged_df)} 行 (1975年之后)")
print(f"最终有效国家共{len(merged_df["Countries"].unique())}个，列表: {merged_df["Countries"].unique()}")
merged_df[merged_df["Countries"] == "China"]

执行单位转换...
计算分解因子...
最终有效数据量: 718 行 (1975年之后)
最终有效国家共75个，列表: ['Algeria' 'Argentina' 'Australia' 'Austria' 'Azerbaijan' 'Bangladesh'
 'Belarus' 'Belgium' 'Brazil' 'Bulgaria' 'Canada' 'Chile' 'China'
 'Colombia' 'Croatia' 'Cyprus' 'Czechia' 'Denmark' 'Ecuador' 'Egypt'
 'Estonia' 'Finland' 'France' 'Germany' 'Greece' 'Hong Kong' 'Hungary'
 'Iceland' 'India' 'Indonesia' 'Iran' 'Iraq' 'Ireland' 'Israel' 'Italy'
 'Japan' 'Kazakhstan' 'Kuwait' 'Latvia' 'Lithuania' 'Luxembourg'
 'Malaysia' 'Mexico' 'Morocco' 'Netherlands' 'New Zealand'
 'North Macedonia' 'Norway' 'Oman' 'Pakistan' 'Peru' 'Philippines'
 'Poland' 'Portugal' 'Qatar' 'Romania' 'Russia' 'Saudi Arabia' 'Singapore'
 'Slovakia' 'Slovenia' 'South Africa' 'Spain' 'Sri Lanka' 'Sweden'
 'Switzerland' 'Thailand' 'Turkmenistan' 'USA' 'Ukraine'
 'United Arab Emirates' 'United Kingdom' 'Uzbekistan' 'Venezuela'
 'Vietnam']


,Countries,Year,total_CE,cement_CS,energy_consumption,Population,GDP,cement_production,built_surface,region,total_CE/energy_consumption,energy_consumption/GDP,GDP/Population,built_surface/Population,cement_CS/built_surface,CFP
130,China,1975,1131.3,5.143255,13.25,9.16395,0.163430,30.021700,2.000345e+10,Asia,85.381132,81.074466,0.017834,2.182842e+09,2.571184e-10,219.957978
132,China,1980,1460.9,14.516760,17.38,9.81235,0.191149,95.442703,2.423702e+10,Asia,84.056387,90.923834,0.019480,2.470052e+09,5.989500e-10,100.635406
134,China,1985,1821.7,20.384433,22.14,10.51040,0.309488,142.489700,2.986116e+10,Asia,82.280939,71.537507,0.029446,2.841106e+09,6.826404e-10,89.367216
136,China,1990,2308.8,28.138505,28.58,11.35185,0.360858,203.168000,3.661076e+10,Asia,80.783765,79.200129,0.031788,3.225092e+09,7.685856e-10,82.051267
138,China,1995,3009.2,56.739027,37.27,12.04855,0.734485,445.610000,4.370784e+10,Asia,80.740542,50.743038,0.060960,3.627643e+09,1.298143e-09,53.035805
140,China,2000,3328.0,78.414183,42.48,12.62645,1.211330,583.190000,5.227580e+10,Asia,78.342750,35.068891,0.095936,4.140182e+09,1.500009e-09,42.441302
142,China,2005,6079.3,132.202206,75.70,13.03720,2.285960,1038.300000,6.022436e+10,Asia,80.307794,33.115190,0.175341,4.619424e+09,2.195162e-09,45.984860
144,China,2010,8121.7,242.079586,104.60,13.37705,6.087190,1880.000000,6.996591e+10,Asia,77.645315,17.183627,0.455047,5.230294e+09,3.459965e-09,33.549710
146,China,2015,9171.3,309.104196,126.49,13.79860,11.061600,2359.188300,8.057881e+10,Asia,72.506127,11.435055,0.801647,5.839637e+09,3.836048e-09,29.670577
148,China,2020,10130.9,359.195176,149.45,14.11100,14.687700,2394.708300,9.012522e+10,Asia,67.787889,10.175181,1.040869,6.386877e+09,3.985512e-09,28.204443


### 输出中间因素表

In [12]:
merged_df.to_csv("Data1/results/factors.csv")

## 3.LMDI分解

#### 注意：因为数据本来就是5年interval，所以也是按5年interval分解的

total_CE/cement_CS = <br>
   (total_CE/energy_consumption) × <br>
   (energy_consumption/GDP) × <br>
   (GDP/Population) × <br>
    (1/(built_surface/Population)) ×  # 倒数关系 <br>
    (1/(cement_CS/built_surface))     # 倒数关系

$ CFP = $
$ \frac{\text{total\_CE}}{\text{cement\_CS}} = $
$ \underbrace{\left( \frac{\text{total\_CE}}{\text{energy\_consumption}} \right)}_{①\text{能源结构}} $
$ \times \underbrace{\left( \frac{\text{energy\_consumption}}{\text{GDP}} \right)}_{②\text{能源效率}} $ 
$ \times \underbrace{\left( \frac{\text{GDP}}{\text{Population}} \right)}_{③\text{经济发展}} $
$ \times \underbrace{\left( \frac{\text{built\_surface}}{\text{Population}} \right) ^ {-1}}_{④\text{建筑密集度}} $
$ \times \underbrace{\left( \frac{\text{cement\_CS}}{\text{built\_surface}} \right) ^ {-1}}_{⑤\text{水泥碳汇强度}} $

$ CFP = $
$ \frac{\text{total\_CE}}{\text{cement\_CS}} = $
$ \left( \frac{\text{total\_CE}}{\text{energy\_consumption}} \right) $
$ \times \left( \frac{\text{energy\_consumption}}{\text{GDP}} \right) $ 
$ \times \left( \frac{\text{GDP}}{\text{Population}} \right) $
$ \times \left( \frac{\text{built\_surface}}{\text{Population}} \right) ^ {-1} $
$ \times \left( \frac{\text{cement\_CS}}{\text{built\_surface}} \right) ^ {-1} $

In [13]:
def lmdi_decomposition(df, base_year, target_ratio='total_CE/cement_CS'):
    """
    执行 LMDI 分解，计算各因子贡献
    - df: 合并后的数据（包含 Country, Year, region 和各因子）
    - base_year: 基期年份（如 1990）
    - target_ratio: 分解的目标比例（默认为 CS/CE）
    """
    # 计算目标比例 CFP = CE/CS
    df[target_ratio] = df['total_CE'] / df['cement_CS']

    # 初始化结果存储
    results = []
    
    for country, country_df in df.groupby('Countries'):
        country_df = country_df.sort_values('Year')
        country_df = country_df[country_df['Year'] >= base_year]
        if len(country_df) < 2:
            continue

        # 获取该国家的 region（假设同一国家的 region 相同，取第一个非空值）
        region = country_df['region'].dropna().iloc[0] if not country_df['region'].isnull().all() else 'Unknown'

        for idx in range(1, len(country_df)):
            t_row = country_df.iloc[idx]
            b_row = country_df.iloc[idx-1]

            cfp_t = t_row[target_ratio]
            cfp_b = b_row[target_ratio]

            # 添加对 CFP 值的保护
            if cfp_t <= 0 or cfp_b <= 0:
                continue  # 跳过非正值

            # 计算 L(CFP^t, CFP^b)
            if np.isclose(cfp_t, cfp_b, rtol=1e-6):  # 浮点数精度容错
                L = cfp_t
            else:
                log_diff = np.log(cfp_t) - np.log(cfp_b)
                L = (cfp_t - cfp_b) / log_diff

            # 计算各因子贡献（添加对 ratio_t/ratio_b 的保护）
            contributions = {}
            for factor in factors:
                ratio_t = t_row[factor]
                ratio_b = b_row[factor]
                if ratio_t <= 0 or ratio_b <= 0:
                    contributions[f'Δ{factor}'] = np.nan
                else:
                    # 对建筑面积/人口项取负号
                    if factor in ['built_surface/Population', 'cement_CS/built_surface']:
                        contributions[f'Δ{factor}'] = -L * np.log(ratio_t / ratio_b)
                    else:
                        contributions[f'Δ{factor}'] = L * np.log(ratio_t / ratio_b)

            results.append({
                'Countries': country,
                'Region': region,  # 添加 region 信息
                'Start_Year': b_row['Year'],
                'End_Year': t_row['Year'],
                'ΔTotal': cfp_t - cfp_b,
                **contributions
            })
        
    return pd.DataFrame(results)

In [14]:
# 设置基期年份（根据数据实际起始时间调整）
base_year = 1975  # 示例

# 运行分解
lmdi_results_5 = lmdi_decomposition(merged_df, base_year)

# 保存结果
lmdi_results_5.to_csv('Data1/results/LMDI_Contributions_5yrs_interval.csv', index=False, encoding='utf-8-sig')

# 查看示例输出
lmdi_results_5.head()

,Countries,Region,Start_Year,End_Year,ΔTotal,Δtotal_CE/energy_consumption,Δenergy_consumption/GDP,ΔGDP/Population,Δbuilt_surface/Population,Δcement_CS/built_surface
0,Algeria,Africa,1975,1980,-26.735148,-2.599560,-5.277985,71.118306,7.890033,-97.865940
1,Algeria,Africa,1980,1985,1.662086,1.370299,5.265836,10.920952,6.494850,-22.389851
2,Algeria,Africa,1985,1990,-6.010295,-4.113439,8.324942,-5.315408,4.876664,-9.783054
3,Algeria,Africa,1990,1995,-13.021165,-0.190956,24.681683,-31.524017,0.915694,-6.903569
4,Algeria,Africa,1995,2000,-9.595745,0.233208,-16.189702,9.912027,-0.704355,-2.846923


In [15]:
# 验证：
lmdi_results_5['ΔSum'] = lmdi_results_5[[f'Δ{factor}' for factor in factors]].sum(axis=1)
print(lmdi_results_5[['ΔTotal', 'ΔSum']].head())

      ΔTotal       ΔSum
0 -26.735148 -26.735148
1   1.662086   1.662086
2  -6.010295  -6.010295
3 -13.021165 -13.021165
4  -9.595745  -9.595745


## 分解20年

In [16]:
def lmdi_decomposition_single_period(df, base_year, target_ratio='cement_CS/total_CE'):
    """
    执行LMDI分解（每个国家只计算最新一年与最开始一年的对比）
    - df: 合并后的数据（包含Country, Year, region和各因子）
    - base_year: 基期年份（如1975）
    - target_ratio: 分解的目标比例（默认为CE/CS）
    """ 
    # 计算目标比例 CFP = CE/CS
    df[target_ratio] = df['total_CE'] / df['cement_CS']
    
    # 初始化结果存储
    results = []
    
    for country, country_df in df.groupby('Countries'):
        # 筛选基期之后的数据并按年份排序
        country_df = country_df[country_df['Year'] >= base_year].sort_values('Year')
        if len(country_df) < 2:
            continue  # 至少需要首尾两年数据
        
        # 获取该国家的region（取第一个非空值）
        region = country_df['region'].dropna().iloc[0] if not country_df['region'].isnull().all() else 'Unknown'
        
        # 获取首尾两年数据
        first_row = country_df.iloc[0]    # 最开始一年（>= base_year的最早年份）
        last_row = country_df.iloc[-1]    # 最新一年
        
        cfp_first = first_row[target_ratio]
        cfp_last = last_row[target_ratio]

        # 跳过非正值
        if cfp_first <= 0 or cfp_last <= 0:
            continue

        # 计算L(CFP_last, CFP_first)
        if np.isclose(cfp_last, cfp_first, rtol=1e-6):
            L = cfp_last
        else:
            log_diff = np.log(cfp_last) - np.log(cfp_first)
            L = (cfp_last - cfp_first) / log_diff if not np.isclose(log_diff, 0) else cfp_last

        # 计算各因子贡献
        contributions = {}
        for factor in factors:
            ratio_first = first_row[factor]
            ratio_last = last_row[factor]
            if ratio_first <= 0 or ratio_last <= 0:
                contributions[f'Δ{factor}'] = np.nan
            else:
                # 对特定因子取负号
                if factor in ['built_surface/Population', 'cement_CS/built_surface']:
                    contributions[f'Δ{factor}'] = -L * np.log(ratio_last / ratio_first)
                else:
                    contributions[f'Δ{factor}'] = L * np.log(ratio_last / ratio_first)
        
        # 只保存没有NaN的结果
        if not np.isnan(list(contributions.values())).any():
            results.append({
                'Countries': country,
                'Region': region,  # 添加region信息
                'Start_Year': first_row['Year'],
                'End_Year': last_row['Year'],
                'Period_Length': last_row['Year'] - first_row['Year'],  # 新增：计算时期长度
                'ΔTotal': cfp_last - cfp_first,  # 总变化量
                **contributions
            })
    
    return pd.DataFrame(results)

In [17]:
# 运行分解（每个国家输出首尾年对比）
lmdi_results_ttl = lmdi_decomposition_single_period(merged_df, base_year=1975)

# 保存结果
lmdi_results_ttl.to_csv('Data1/results/LMDI_Contributions_total_yrs_interval.csv', index=False)

# 查看示例输出
print(lmdi_results_ttl.head())

    Countries    Region  Start_Year  End_Year  Period_Length      ΔTotal  \
0     Algeria    Africa        1975      2020             45  -61.823809   
1   Argentina  Americas        1975      2020             45 -375.147644   
2   Australia   Oceania        1975      2020             45  -38.299867   
3     Austria    Europe        1975      2020             45  -22.924944   
4  Azerbaijan      Asia        1975      2020             45   63.085636   

   Δtotal_CE/energy_consumption  Δenergy_consumption/GDP  ΔGDP/Population  \
0                     -5.742591                -7.993874        86.542818   
1                    -45.790970            -10485.738756     10530.259252   
2                    -17.083003              -290.943671       317.892709   
3                    -12.691645               -83.850003        91.783881   
4                      9.879642               -61.845422       116.595719   

   Δbuilt_surface/Population  Δcement_CS/built_surface  
0                  18.2

## 4.计算单位CS变化产生的贡献变化率
### 根据tapio解耦思路，计算因素变化量，用来衡量单位因素变化的贡献变化量

#### tapio弹性? tapio elasticity

#### $ T = \frac{(C_5^T - C_5^0) / C_5^0}{(CS^T - CS^0)/CS^0} $



In [18]:
def calc_grad(attr, df = merged_df):
    res = []
    for country, country_df in df.groupby('Countries'):
        country_df = country_df.sort_values('Year')

        if len(country_df) < 2:
            continue

        for idx in range(1, len(country_df)):
            current_row = country_df.iloc[idx]  # current interval
            previous_row = country_df.iloc[idx-1]  # previous interval
    
            current_val = current_row[attr]
            previous_val = previous_row[attr]
        
            # 处理分母为0的情况
            if previous_val == 0:
                if current_val > 0:
                    grad = np.inf
                elif current_val < 0:
                    grad = -np.inf
                else:
                    grad = 0  # 0/0的情况
            else:
                grad = (current_val - previous_val) / previous_val
            
            res.append({
                'Countries': country,
                'Interval': f"{idx}",
                'Years': f"{previous_row['Year']} - {current_row['Year']}",
                f'grad({attr})': grad
            })

    return pd.DataFrame(res)

#### 4.1 计算 $ (C_5^T - C_5^0) / C_5^0 $

In [19]:
C5_grad_results = calc_grad("cement_CS/built_surface")
C5_grad_results

,Countries,Interval,Years,grad(cement_CS/built_surface)
0,Algeria,1,1975 - 1980,2.115938
1,Algeria,2,1980 - 1985,0.351891
2,Algeria,3,1985 - 1990,0.145437
3,Algeria,4,1990 - 1995,0.117091
4,Algeria,5,1995 - 2000,0.057276
...,...,...,...,...
638,Vietnam,5,1995 - 2000,0.269070
639,Vietnam,6,2000 - 2005,1.179654
640,Vietnam,7,2005 - 2010,0.306802
641,Vietnam,8,2010 - 2015,-0.016621


#### 4.2 计算 $ (CS^T - CS^0)/CS^0 $

In [20]:
CS_grad_results = calc_grad("cement_CS")
CS_grad_results

,Countries,Interval,Years,grad(cement_CS)
0,Algeria,1,1975 - 1980,2.388194
1,Algeria,2,1980 - 1985,0.462990
2,Algeria,3,1985 - 1990,0.234197
3,Algeria,4,1990 - 1995,0.228491
4,Algeria,5,1995 - 2000,0.158393
...,...,...,...,...
638,Vietnam,5,1995 - 2000,0.645935
639,Vietnam,6,2000 - 2005,1.581300
640,Vietnam,7,2005 - 2010,0.549556
641,Vietnam,8,2010 - 2015,0.138590


In [21]:
# 合并数据时指定只保留一个Years列
Tapio_results_df = pd.merge(
    C5_grad_results,
    CS_grad_results, 
    on=['Countries', 'Interval', 'Years'],  # 将Years也作为合并键
    how='inner'
)

# 选择需要的列
Tapio_results_df = Tapio_results_df[['Countries', 'Years', 'grad(cement_CS/built_surface)', 'grad(cement_CS)']]

# 将grad(built_surface/cement_CS)转换为百分数形式（乘以100）
# Tapio_results_df["grad(built_surface/cement_CS)"] = Tapio_results_df["grad(built_surface/cement_CS)"] * 100

# 计算Tapio弹性
Tapio_results_df["CS_tapio_elasticity"] = Tapio_results_df["grad(cement_CS/built_surface)"] / Tapio_results_df["grad(cement_CS)"]

In [22]:
Tapio_results_df.to_csv('Data1/results/C5_CS_Tapio_results_5_yrs_interval.csv', index=False)
Tapio_results_df

,Countries,Years,grad(cement_CS/built_surface),grad(cement_CS),CS_tapio_elasticity
0,Algeria,1975 - 1980,2.115938,2.388194,0.885999
1,Algeria,1980 - 1985,0.351891,0.462990,0.760041
2,Algeria,1985 - 1990,0.145437,0.234197,0.621001
3,Algeria,1990 - 1995,0.117091,0.228491,0.512453
4,Algeria,1995 - 2000,0.057276,0.158393,0.361610
...,...,...,...,...,...
638,Vietnam,1995 - 2000,0.269070,0.645935,0.416559
639,Vietnam,2000 - 2005,1.179654,1.581300,0.746003
640,Vietnam,2005 - 2010,0.306802,0.549556,0.558273
641,Vietnam,2010 - 2015,-0.016621,0.138590,-0.119932


### 4.5 计算分区域的

In [30]:
def calc_grad(attr, df):
    res = []
    for region, region_df in df.groupby('region'):
        region_df = region_df.sort_values('Year')

        if len(region_df) < 2:
            continue

        for idx in range(1, len(region_df)):
            current_row = region_df.iloc[idx]  # current interval
            previous_row = region_df.iloc[idx-1]  # previous interval
    
            current_val = current_row[attr]
            previous_val = previous_row[attr]
        
            # Handle division by zero
            if previous_val == 0:
                if current_val > 0:
                    grad = np.inf
                elif current_val < 0:
                    grad = -np.inf
                else:
                    grad = 0  # 0/0 case
            else:
                grad = (current_val - previous_val) / previous_val
            
            res.append({
                'region': region,
                'Interval': f"{idx}",
                'Years': f"{previous_row['Year']} - {current_row['Year']}",
                f'grad({attr})': grad
            })

    return pd.DataFrame(res)

region_factors_df = pd.read_csv('Data1/results/Factors_by_Region_and_Year_5yrs_interval.csv')

region_C5_grad_results = calc_grad("cement_CS/built_surface", region_factors_df)
region_CS_grad_results = calc_grad("cement_CS", region_factors_df)

# Then merge them (this part needs adjustment based on your actual metrics)
Tapio_results_df = pd.merge(
    region_C5_grad_results,
    region_CS_grad_results, 
    on=['region', 'Interval', 'Years'],
    how='inner'
)

# Select needed columns
Region_Tapio_results_df = Tapio_results_df[['region', 'Years', 'grad(cement_CS/built_surface)', 'grad(cement_CS)']]

# Calculate Tapio elasticity
Region_Tapio_results_df["tapio_elasticity"] = Tapio_results_df["grad(cement_CS/built_surface)"] / Tapio_results_df["grad(cement_CS)"]

Region_Tapio_results_df.to_csv('Data1/results/Region_C5_CS_Tapio_results_5_yrs_interval.csv', index=False)
Region_Tapio_results_df

,region,Years,grad(cement_CS/built_surface),grad(cement_CS),tapio_elasticity
0,Africa,1975 - 1980,0.276437,0.261867,1.055640
1,Africa,1980 - 1985,0.210337,0.363952,0.577924
2,Africa,1985 - 1990,0.136788,0.253652,0.539277
3,Africa,1990 - 1995,0.204626,0.310872,0.658232
4,Africa,1995 - 2000,0.116112,0.231313,0.501971
5,Africa,2000 - 2005,0.129787,0.161903,0.801633
6,Africa,2005 - 2010,0.438588,0.576550,0.760711
7,Africa,2010 - 2015,0.012553,0.148393,0.084592
8,Africa,2015 - 2020,-0.183108,-0.072698,2.518730
9,Americas,1975 - 1980,0.340066,0.453817,0.749346


# 计算整体tapio

# 基于平均值的Tapio弹性系数计算方法

## 方法定义

基于平均值基准的Tapio弹性系数计算公式：

$$
E_{\text{avg}} = \frac{\%\Delta F5_{\text{avg}}}{\%\Delta CS_{\text{avg}}}
$$

其中：
- $E_{\text{avg}}$：基于平均值的Tapio弹性系数
- $\%\Delta F5_{\text{avg}}$：单位面积碳封存效率($F5$)相对于平均值的百分比变化
- $\%\Delta CS_{\text{avg}}$：总水泥碳封存量($CS$)相对于平均值的百分比变化

## 公式推导

### 变量定义
$$
\begin{aligned}
F5_t &= \frac{CS_t}{A_t} \quad &\text{(单位面积碳封存效率)} \\
\overline{F5} &= \frac{1}{T}\sum_{t=1}^{T} F5_t \quad &\text{($F5$的时间平均值)} \\
\overline{CS} &= \frac{1}{T}\sum_{t=1}^{T} CS_t \quad &\text{($CS$的时间平均值)}
\end{aligned}
$$

### 变化量计算
研究期间从$t_0$到$t_T$的变化：
$$
\begin{aligned}
\Delta F5 &= F5_{t_T} - F5_{t_0} \\
\Delta CS &= CS_{t_T} - CS_{t_0}
\end{aligned}
$$

### 相对于平均值的变化率
$$
\begin{aligned}
\%\Delta F5_{\text{avg}} &= \frac{\Delta F5}{\overline{F5}} = \frac{F5_{t_T} - F5_{t_0}}{\overline{F5}} \\
\%\Delta CS_{\text{avg}} &= \frac{\Delta CS}{\overline{CS}} = \frac{CS_{t_T} - CS_{t_0}}{\overline{CS}}
\end{aligned}
$$

### 最终弹性系数公式
$$
E_{\text{avg}} = \frac{(F5_{t_T} - F5_{t_0})/\overline{F5}}{(CS_{t_T} - CS_{t_0})/\overline{CS}} = \frac{\Delta F5 \cdot \overline{CS}}{\Delta CS \cdot \overline{F5}}
$$

## 方法优势对比

| 特征 | 传统基期方法 | 平均值基准方法 |
|------|--------------|----------------|
| **基准点** | 起始年$t_0$值 | 整个时段平均值 |
| **稳定性** | 受起始年异常值影响大 | 对异常值鲁棒 |
| **数学表达** | $\frac{\Delta F5/F5_0}{\Delta CS/CS_0}$ | $\frac{\Delta F5/\overline{F5}}{\Delta CS/\overline{CS}}$ |
| **适用场景** | 短期变化分析 | 长期趋势分析 |

## 弹性系数解读

弹性系数$E_{\text{avg}}$的经济环境意义：

- **$E_{\text{avg}} > 1$**  
  单位面积碳封存效率提升快于总量增长（最佳情景）
  
- **$0 < E_{\text{avg}} < 1$**  
  效率提升但慢于总量增长（粗放增长）

- **$E_{\text{avg}} < 0$**  
  效率与总量呈反向变化（需预警）

- **$E_{\text{avg}} = \text{NaN}$**  
  平均值或变化量为零（需特殊处理）

## 计算示例（伪代码）

```python
def calc_avg_tapio(df):
    avg_F5 = df['F5'].mean()
    avg_CS = df['CS'].mean()
    delta_F5 = df['F5'].iloc[-1] - df['F5'].iloc[0]
    delta_CS = df['CS'].iloc[-1] - df['CS'].iloc[0]
    return (delta_F5/avg_F5) / (delta_CS/avg_CS)

In [23]:
import pandas as pd
import numpy as np

def calc_avg_based_total_tapio(df):
    """计算每个国家基于平均值（而非起始值）的总Tapio弹性系数"""
    results = []
    
    for country, country_df in df.groupby('Countries'):
        # 按年份排序
        country_df = country_df.sort_values('Year')
        
        # 确保有足够数据点
        if len(country_df) < 2:
            continue
            
        # 计算整个时间段的平均值
        avg_F5 = np.mean(country_df['cement_CS'] / country_df['built_surface'])
        avg_CS = np.mean(country_df['cement_CS'])
        
        # 获取起始年和结束年数据
        start_row = country_df.iloc[0]
        end_row = country_df.iloc[-1]
        
        # 计算起始年和结束年的F5值
        F5_start = start_row['cement_CS'] / start_row['built_surface']
        F5_end = end_row['cement_CS'] / end_row['built_surface']
        
        # 获取起始年和结束年的cement_CS值
        CS_start = start_row['cement_CS']
        CS_end = end_row['cement_CS']
        
        # 计算相对于平均值的变化量
        delta_F5 = (F5_end - F5_start)  # 绝对值变化
        delta_CS = (CS_end - CS_start)   # 绝对值变化
        
        # 计算相对于平均值的百分比变化
        if avg_F5 == 0:
            pct_delta_F5 = np.nan
        else:
            pct_delta_F5 = delta_F5 / avg_F5  # 相对于平均值的百分比变化
        
        if avg_CS == 0:
            pct_delta_CS = np.nan
        else:
            pct_delta_CS = delta_CS / avg_CS  # 相对于平均值的百分比变化
        
        # 计算Tapio弹性系数
        if pct_delta_CS == 0 or np.isnan(pct_delta_F5) or np.isnan(pct_delta_CS):
            tapio_elasticity = np.nan
        else:
            tapio_elasticity = pct_delta_F5 / pct_delta_CS
        
        results.append({
            'Country': country,
            'Start_Year': start_row['Year'],
            'End_Year': end_row['Year'],
            # 'F5_Avg': avg_F5,
            # 'CS_Avg': avg_CS,
            # 'F5_Start': F5_start,
            # 'F5_End': F5_end,
            # 'CS_Start': CS_start,
            'CS_End': CS_end,
            'Delta_F5': delta_F5,
            'Delta_CS': delta_CS,
            'Pct_Delta_F5': pct_delta_F5,
            'Pct_Delta_CS': pct_delta_CS,
            'Total_Tapio_Elasticity': tapio_elasticity
        })
    
    return pd.DataFrame(results)

# 使用函数计算基于平均值的总Tapio弹性系数
avg_based_tapio_results = calc_avg_based_total_tapio(merged_df)
avg_based_tapio_results

,Country,Start_Year,End_Year,CS_End,Delta_F5,Delta_CS,Pct_Delta_F5,Pct_Delta_CS,Total_Tapio_Elasticity
0,Algeria,1975,2020,3.460795,1.282208e-09,3.306055,1.543416,2.011264,0.767386
1,Argentina,1975,2020,2.058820,3.991993e-10,1.853744,0.937408,1.219498,0.768684
2,Australia,1975,2020,2.685450,6.340597e-11,1.611499,0.158331,0.999376,0.158430
3,Austria,1975,2020,1.819373,-5.133778e-11,0.823363,-0.033017,0.557582,-0.059214
4,Azerbaijan,1975,2020,0.557972,2.339809e-10,0.389054,0.499571,1.218772,0.409897
...,...,...,...,...,...,...,...,...,...
69,United Arab Emirates,1980,2020,3.086787,4.390132e-09,3.050516,1.234087,1.819386,0.678299
70,United Kingdom,1975,2020,4.890290,-6.528528e-11,1.515382,-0.082502,0.384074,-0.214806
71,Uzbekistan,1975,2020,2.682688,-1.742475e-10,1.436916,-0.322058,1.019514,-0.315894
72,Venezuela,1975,2020,0.653176,-5.296497e-10,-0.069470,-0.562022,-0.064709,8.685332


In [24]:
avg_based_tapio_results.to_csv("Data1/results/C5_CS_Tapio_results_35yrs_interval.csv")

## 计算regions整体Tapio

In [37]:
import pandas as pd
import numpy as np

def calc_avg_based_total_tapio(df, carbon_col="cement_CS", activity_col="built_surface"):
    """
    计算每个区域基于平均值（而非起始值）的总Tapio弹性系数
    严格遵循原始公式：
        E_avg = [ (F5_end - F5_start) / avg_F5 ] / [ (CS_end - CS_start) / avg_CS ]
              = (pct_delta_F5) / (pct_delta_CS)
    其中：
        F5 = carbon_col / activity_col (碳吸收强度)
        CS = carbon_col (总碳吸收)
    """
    results = []
    
    for region, region_df in df.groupby('region'):
        # 按年份排序
        region_df = region_df.sort_values('Year')
        
        # 确保有足够数据点
        if len(region_df) < 2:
            continue
            
        '''计算整个时间段的平均值'''
        # F5: 碳吸收强度 (碳吸收量/活动量)
        F5_values = region_df[carbon_col] / region_df[activity_col]
        avg_F5 = np.mean(F5_values)
        # CS: 总碳吸收量
        avg_CS = np.mean(region_df[carbon_col])

        '''计算整个时间段的变化量'''
        # 获取起始年和结束年数据
        start_row = region_df.iloc[0]
        end_row = region_df.iloc[-1]
        # 计算起始年和结束年的F5值
        F5_start = start_row[carbon_col] / start_row[activity_col]
        F5_end = end_row[carbon_col] / end_row[activity_col]
        # 获取起始年和结束年的碳吸收值
        CS_start = start_row[carbon_col]
        CS_end = end_row[carbon_col]
        # 计算绝对值变化量
        delta_F5 = F5_end - F5_start
        delta_CS = CS_end - CS_start

        '''计算变化率'''
        if avg_F5 == 0:
            pct_delta_F5 = np.nan
        else:
            pct_delta_F5 = delta_F5 / avg_F5  # 相对于平均值的百分比变化
        
        if avg_CS == 0:
            pct_delta_CS = np.nan
        else:
            pct_delta_CS = delta_CS / avg_CS  # 相对于平均值的百分比变化
        
        '''计算Tapio弹性系数'''
        if pct_delta_CS == 0 or np.isnan(pct_delta_F5) or np.isnan(pct_delta_CS):
            tapio_elasticity = np.nan
        else:
            tapio_elasticity = pct_delta_F5 / pct_delta_CS
        
        results.append({
            'Region': region,
            'Start_Year': start_row['Year'],
            'End_Year': end_row['Year'],
            'CS_End': CS_end,
            'Delta_F5': delta_F5,
            'Delta_CS': delta_CS,
            'Pct_Delta_F5': pct_delta_F5,
            'Pct_Delta_CS': pct_delta_CS,
            'Total_Tapio_Elasticity': tapio_elasticity
        })
    
    return pd.DataFrame(results)

# 使用示例:
# 假设您的数据框名为 region_df
# 确保数据框中包含必要的列：region, Year, total_CE, built_surface（或其他活动量指标）

region_factors_df = pd.read_csv('Data1/results/Factors_by_Region_and_Year_5yrs_interval.csv')
# 计算基于平均值的总Tapio弹性系数
region_factors_tapio_results_35 = calc_avg_based_total_tapio(region_factors_df, 
                                          carbon_col="total_CE", 
                                          activity_col="built_surface")

# 保存结果
region_factors_tapio_results_35.to_csv('Data1/results/Region_total_C5_CS_Tapio_results_35yrs_interval.csv', index=False)
region_factors_tapio_results_35

,Region,Start_Year,End_Year,CS_End,Delta_F5,Delta_CS,Pct_Delta_F5,Pct_Delta_CS,Total_Tapio_Elasticity
0,Africa,1975,2020,844.6,1.800514e-08,618.8,0.284103,1.071163,0.265228
1,Americas,1975,2020,1735.8,-1.199085e-10,892.0,-0.002740,0.635717,-0.004310
2,Asia,1975,2020,16670.9,3.842827e-08,14084.5,0.568258,1.636595,0.347220
3,Europe,1975,2020,2679.5,-5.872573e-08,-743.7,-0.727181,-0.201639,3.606345
4,Oceania,1975,2020,410.1,-5.269031e-09,198.8,-0.072727,0.577974,-0.125831
